DCGAN or Deep Convolutional GAN, is a generative adversarial network architecture. It uses a couple of guidelines. In particular:



*   Replacing any pooling layers with the strided convolutions (discriminator) and fractional-strided convolutions (generators)

*   Using bathnorm in both generator and discriminator
*   Removing fully connected hiiden layers for deeper architecture


*   Using ReLU activation in generator for all lyers except for the output which uses tanh


*   Using LeakyReLu activation in the discriminator for all layers

this is the source : https://paperswithcode.com/method/dcgan#:~:text=DCGAN%2C%20or%20Deep%20Convolutional%20GAN,the%20generator%20and%20the%20discriminator.



In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Input, Flatten,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Dropout, Reshape, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

In [2]:
img_dims = 28
img_chn1 = 1
ltnt_dim = 100

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [5]:
# We need to make sure that each image has a third dimension
x_train = np.expand_dims(x_train, axis=3) # 28x28x1
x_test = np.expand_dims(x_test, axis=3)

In [6]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

x_train shape:  (60000, 28, 28, 1)
x_test shape:  (10000, 28, 28, 1)


In [11]:
# Building the generator network
inpt_noise = Input(shape=(ltnt_dim))

g11 = Dense(7*7*256, activation='relu')(inpt_noise)
g12 = BatchNormalization()(g11)
g13 = Reshape((7,7,256))(g12)
g14= Conv2DTranspose(128, (5,5), strides=(1,1),padding='same', activation='relu')(g13)
g15 = BatchNormalization()(g14)
g16= Conv2DTranspose(64, (5,5), strides=(2,2),padding='same', activation='relu')(g15)
g17 = BatchNormalization()(g16)
g18= Conv2DTranspose(1, (5,5), strides=(2,2),padding='same', activation='sigmoid')(g17)
generator = Model(inpt_noise, g18)
gnrtr_img = generator(inpt_noise)
generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 dense_2 (Dense)             (None, 12544)             1266944   
                                                                 
 batch_normalization_2 (Batc  (None, 12544)            50176     
 hNormalization)                                                 
                                                                 
 reshape_2 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 7, 7, 128)        819328    
 nspose)                                                         
                                                                 
 batch_normalization_3 (Batc  (None, 7, 7, 128)        512   

In [12]:
# Building the  discriminator.
inpt_img = Input(shape=(img_dims,img_dims, img_chn1 ))
d11 = Conv2D(64,(5,5), strides=(2,2), padding='same', activation='relu')(inpt_img)
d12= Dropout(0.3)(d11)

d13 = Conv2D(128,(5,5), strides=(2,2), padding='same', activation='relu')(d12)
d14= Dropout(0.3)(d13)

d15 = Flatten()(d14)
d16= Dense(1,activation='sigmoid')(d15)
critic = Model(inpt_img, d16)
validity = critic(gnrtr_img)
critic.summary() # critic = discriminator

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 64)        1664      
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense_3 (Dense)             (None, 1)                 6273

In [13]:
# We will put things together
optimizer = Adam(0.0002, 0.5)
# Compiling the discriminator
critic.compile(loss='binary_crossentropy', optimizer=optimizer, 
               metrics=['accuracy'])
# We will freeze the discriminator 
critic.trainable = False
gen_critic = Model(inpt_noise, validity) # full model
gen_critic.compile(loss='binary_crossentropy', optimizer=optimizer)
# Creating some trainable parameters
epochs = 12001
batch_size = 128 # small batches is highly recommended
sample_interval = 200 # generatoing samples